In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import gspread

In [28]:
# Define paths

ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data" / "raw"
SECRETS_DIR = ROOT_DIR / "secrets"
SECRETS_FILE = SECRETS_DIR / "secrets.json"

DOC_GSHEET = "Cacciamani Workers Liuna PR 55"
DOC_SHEET = 0
CALLS_GSHEET = "cacciamani-20250912"
CALLS_SHEET = 1

In [8]:
base = Path(".")

print("Relative dot:", base)
print("Resolved dot:", base.resolve())

example1 = "pless-plase"
print("\nBefore resolve:", example1)
print("After resolve: ", example1.resolve())

example2 = Path("..\\other-project\\notes.txt")
print("\nBefore resolve:", example2)
print("After resolve: ", example2.resolve())


Relative dot: .
Resolved dot: D:\my-projects\work\google-access\projects\cacciamani-request\notebooks

Before resolve: pless-plase


AttributeError: 'str' object has no attribute 'resolve'

In [30]:
# Get data

# --- Authentication ---
# Use the JSON key you downloaded from the Google Cloud Console.
# Make sure you have shared your Google Sheet with the service account's email.

def get_data_from_gsheet(cred_file_path: Path, file_name: str, sheet: int = 0) -> pd.DataFrame:
    """Fetch data from a Google Sheet and return it as a pandas DataFrame."""
    try:
        # The scope defines the permissions your script is requesting.
        scope = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        # Authenticate using the service account credentials
        # Replace 'path/to/your/credentials.json' with the actual path to your key file.
        gc = gspread.service_account(cred_file_path, scopes=scope)
        print("Authentication successful!")

    except Exception as e:
        print(f"Authentication failed: {e}")
        # Exit or handle the error appropriately
        exit()

    # --- Open the Google Sheet ---
    try:
        # Open the spreadsheet by its title.
        # Replace 'Your Google Sheet Name' with the exact name of your sheet.
        spreadsheet = gc.open(file_name)
        print(f"Successfully opened spreadsheet: '{spreadsheet.title}'")

        # Select the first worksheet. You can also select by name.
        # For example: worksheet = spreadsheet.worksheet('Sheet1')
        worksheet = spreadsheet.get_worksheet(sheet)
        print(f"Using worksheet: '{worksheet.title}'")

        # --- Load data into a pandas DataFrame ---
        # get_all_records() returns a list of dictionaries, which is perfect for a DataFrame.
        # It automatically uses the first row as headers.
        data = worksheet.get_all_records()
        
        if not data:
            print("Worksheet is empty.")
            return pd.DataFrame()
        else:
            df = pd.DataFrame(data)
            print("Data loaded into DataFrame successfully:")
            print(df.head())
            return df

    except gspread.exceptions.SpreadsheetNotFound:
        print("Error: Spreadsheet not found. Make sure the name is correct and you have shared it with the service account.")
    except Exception as e:
        print(f"An error occurred: {e}")

    # Now you can work with your DataFrame 'df'
    # For example:
    # print(df.describe())


In [31]:
df = get_data_from_gsheet(SECRETS_FILE, DOC_GSHEET, DOC_SHEET)
df.head()

Authentication successful!
Error: Spreadsheet not found. Make sure the name is correct and you have shared it with the service account.


AttributeError: 'NoneType' object has no attribute 'head'